## DỊCH CÂU HỎI ĐẦU VÀO SANG TIẾNG VIỆT

In [4]:
# !pip install google-cloud-translate

In [5]:
from google.cloud import translate_v2 as translate
import json
from collections import OrderedDict
import os
import re

# Thiết lập Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"googleKey.json"

# Khởi tạo Google Translate API client
translate_client = translate.Client()


In [6]:
# def protect_keywords(text):
#     """
#     Bảo vệ các từ khóa logic (AND, JOIN, R) bằng cách thay thế chúng bằng placeholder độc nhất.
#     """
#     if not text:
#         return text
    
#     protected_text = re.sub(r'\b(AND|JOIN|R)\b', r'__\1__', text, flags=re.IGNORECASE)
#     return protected_text

# def restore_keywords(text):
#     """
#     Khôi phục các từ khóa logic từ placeholder.
#     """
#     if not text:
#         return text
    
#     restored_text = re.sub(r'__(AND|JOIN|R)__', r'\1', text, flags=re.IGNORECASE)
#     return restored_text

In [7]:
def translate_to_vietnamese(input_text):
    """
    Dịch văn bản sang tiếng Việt, bảo vệ và khôi phục từ khóa.
    """
    if not input_text:
        return None
    
    try:
        result = translate_client.translate(
            input_text, 
            target_language="vi",
            format_="text"  # Giữ nguyên định dạng
        )
        translated_text = result["translatedText"]
        return translated_text
    
    except Exception as e:
        print(f"Lỗi dịch: {str(e)}")
        return input_text  # Trả lại bản gốc nếu có lỗi


In [8]:
def read_json(file_path):
    """Đọc file JSON với encoding UTF-8"""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(file_path, data):
    """Ghi file JSON với định dạng đẹp"""
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4, separators=(",", ": "))

In [9]:
def process_data(data, output_file):
    """
    Xử lý dữ liệu và lưu file sau mỗi 10 bản ghi.
    """
    processed_data = []
    count = 0
    
    for idx, item in enumerate(data):
        try:
            new_item = OrderedDict()
            field_order = [
                    "question_id","question","question_vi","sparql", "answer","s_expr", "nor_s_expr", "gold_entity_map","gold_relation_map",

            ]
            
            # Dịch các trường cần thiết
            translations = {
                "question_vi": translate_to_vietnamese(item.get("question", "")),
            }
            
            # Xây dựng item mới
            for field in field_order:
                if field in translations:
                    new_item[field] = translations[field]
                elif field in item:
                    new_item[field] = item[field]
            
            processed_data.append(new_item)
            count += 1
            if count % 100 == 0:
                print(f"Đã xử lý: {count} bản ghi")
            
            # Lưu tạm sau mỗi 10 bản ghi
            if count % 10 == 0:
                write_json(output_file, processed_data)
                print(f"Đã lưu tạm {count} bản ghi")
        
        except Exception as e:
            print(f"Lỗi ở bản ghi {idx}: {str(e)}")
            continue
    
    # Lưu phần còn lại
    write_json(output_file, processed_data)
    print(f"✅ Đã lưu tổng cộng {count} bản ghi")
    return processed_data

In [ ]:
# input_file = "filter/test_filtered.json"
# output_file = "translate/test_translated.json"

# try:
#     data = read_json(input_file)
#     print(f"Đã đọc {len(data)} bản ghi từ {input_file}")
    
#     # Xử lý tiếp nếu file output đã tồn tại
#     if os.path.exists(output_file):
#         existing_data = read_json(output_file)
#         start_idx = len(existing_data)
#         print(f"Tiếp tục xử lý từ bản ghi {start_idx}")
#         processed_data = process_data(data[start_idx:], output_file)
#     else:
#         processed_data = process_data(data, output_file)
        
#     print("Hoàn thành dịch cho file: ", input_file )

# except Exception as e:
#     print(f"Lỗi nghiêm trọng: {str(e)}")


In [ ]:
input_file = "filter/train_filtered.json"
output_file = "translate/train_translated.json"

try:
    data = read_json(input_file)
    print(f"Đã đọc {len(data)} bản ghi từ {input_file}")
    
    # Xử lý tiếp nếu file output đã tồn tại
    if os.path.exists(output_file):
        existing_data = read_json(output_file)
        start_idx = len(existing_data)
        print(f"Tiếp tục xử lý từ bản ghi {start_idx}")
        processed_data = process_data(data[start_idx:], output_file)
    else:
        processed_data = process_data(data, output_file)
        
    print("Hoàn thành dịch cho file: ", input_file )

except Exception as e:
    print(f"Lỗi nghiêm trọng: {str(e)}")


Đã đọc 17613 bản ghi từ train_filtered.json
Đã lưu tạm 10 bản ghi
Đã lưu tạm 20 bản ghi
Đã lưu tạm 30 bản ghi
Đã lưu tạm 40 bản ghi
Đã lưu tạm 50 bản ghi
Đã lưu tạm 60 bản ghi
Đã lưu tạm 70 bản ghi
Đã lưu tạm 80 bản ghi
Đã lưu tạm 90 bản ghi
Đã xử lý: 100 bản ghi
Đã lưu tạm 100 bản ghi
Đã lưu tạm 110 bản ghi
Đã lưu tạm 120 bản ghi
Đã lưu tạm 130 bản ghi
Đã lưu tạm 140 bản ghi
Đã lưu tạm 150 bản ghi
Đã lưu tạm 160 bản ghi
Đã lưu tạm 170 bản ghi
Đã lưu tạm 180 bản ghi
Đã lưu tạm 190 bản ghi
Đã xử lý: 200 bản ghi
Đã lưu tạm 200 bản ghi
Đã lưu tạm 210 bản ghi
Đã lưu tạm 220 bản ghi
Đã lưu tạm 230 bản ghi
Đã lưu tạm 240 bản ghi
Đã lưu tạm 250 bản ghi
Đã lưu tạm 260 bản ghi
Đã lưu tạm 270 bản ghi
Đã lưu tạm 280 bản ghi
Đã lưu tạm 290 bản ghi
Đã xử lý: 300 bản ghi
Đã lưu tạm 300 bản ghi
Đã lưu tạm 310 bản ghi
Đã lưu tạm 320 bản ghi
Đã lưu tạm 330 bản ghi
Đã lưu tạm 340 bản ghi
Đã lưu tạm 350 bản ghi
Đã lưu tạm 360 bản ghi
Đã lưu tạm 370 bản ghi
Đã lưu tạm 380 bản ghi
Đã lưu tạm 390 bản ghi
Đã